In [50]:
import json
import numpy as np
import pandas as pd
d1, d2, d3 = 180, 18, 42


In [76]:
k0 = 3
nt = 5*k0
estimate = res["supervised"][k0][1]["estimation"]
Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
Tau = Mu * Lambda
S = 1/Lambda
gene = []
def dist(t1, t2, i):
    mu1, mu2, s1, s2 = Mu[t1,1], Mu[t1,i], S[t1,i], S[t2,i]
    s = (s1+s2)/2
    return (mu1-mu2)**2/s/8+1/2*np.log(s/np.sqrt(s1*s2))

for i in range(240):
    tem = []
    for t1 in range(nt-1):
        for t2 in range (t1+1,nt):
            tem.append(dist(t1,t2,i))
    gene.append(max(tem))



In [78]:
gene1 = gene

In [79]:
k0 = 9
nt = 5*k0
estimate = res["supervised"][k0][1]["estimation"]
Mu, Lambda = [estimate["Mu1"], estimate["Mu2"], estimate["Mu3"]], [estimate["Lambda1"], estimate["Lambda2"], estimate["Lambda3"]]
Mu, Lambda = [np.array(mus) for mus in Mu], [np.array(lambdas) for lambdas in Lambda]
Mu = np.concatenate((Mu[0], Mu[1], Mu[2]), axis=1)
Lambda = np.concatenate((Lambda[0], Lambda[1], Lambda[2]), axis=1)
Tau = Mu * Lambda
S = 1/Lambda
gene = []
def dist(t1, t2, i):
    mu1, mu2, s1, s2 = Mu[t1,1], Mu[t1,i], S[t1,i], S[t2,i]
    s = (s1+s2)/2
    return (mu1-mu2)**2/s/8+1/2*np.log(s/np.sqrt(s1*s2))

for i in range(240):
    tem = []
    for t1 in range(nt-1):
        for t2 in range (t1+1,nt):
            tem.append(dist(t1,t2,i))
    gene.append(max(tem))



In [87]:
gene = np.array(gene)
n = 20
idx = np.argpartition(gene, -n)[-n:]
indices = idx[np.argsort((-gene)[idx])]
indices

array([217, 202, 205, 231, 200, 198, 199, 224, 209, 207, 211, 204, 233,
       235, 218, 234, 230, 223, 201, 206])

In [90]:
print(gene[indices.tolist()])
print(gene[list(range(10))])


[7.53055417 6.84389654 6.28953733 5.78788215 5.46889616 4.92995231
 4.51699618 4.42768504 4.11941815 3.99834579 3.60219404 3.34480437
 3.26045544 3.12172397 2.9064809  2.66039241 2.46078044 2.40228011
 2.26054697 2.20225871]
[0.74039333 0.03482271 0.46393796 0.40341546 0.26484168 0.53092709
 0.59888151 0.80673752 0.80248388 0.98687181]
